# Fundamental Aspects

This document aims to provide an overview of important algorithms and implementations that should help users understand the results one obtains from LobsterPy.

:::{important}
In the following sections the implementations are described using ICOHPs/COHPs. 

It is implied that the procedure followed is the same even for analyzing and curating features for ML for ICOOPs/COOPs or ICOBIs/COBIs.
:::

## LobsterEnv : Detect coordination environments based on the electronic structure

:::{admonition} New in LobsterEnv
:class: versionadded
* Can handle ICOBIs and ICOOPs to detect co-ordination environments and relevant bonds.
* Provide structure graphs with ICOHPs/ ICOBIs/ ICOOPs as edge properties
:::

Lobsterenv is the central algorithm on which LobsterPy depends for automatic bonding analysis. 

Below is a schematic depicting how this algorithm works. The image is from a talk given by Prof. Dr. George at [MC16 2023 Dublin](https://opus4.kobv.de/opus4-bam/frontdoor/index/index/start/0/rows/10/sortfield/year/sortorder/desc/searchtype/simple/query/George/author_facetfq/George%2C+Janine/doctypefq/lecture/docId/57876)  

![Schema depicting working of LobsterEnv algorithm](fundamentals_assets/LobsterEnv_schema.png)

Unlike purely geometry-based algorithms, e.g., Voronoi construction, which depends on angle and distance cutoffs, LobsterEnv needs only one parameter that influences neighbor selection, i.e., ICOHPs.

Thus, when running automatic analysis using the Analysis module, `cutoff_icohp` is the parameter to be paid attention to. This value is set to 10% by default. This means that, depending on the type of bond (cation-anion or all), only ICOHPs with a value of more than 10% of the largest ICOHPs (this value also changes depending on the type of bonds considered for analysis, cation-anion/all bonds) are considered in the analysis.

The `noise_cutoff` parameter takes care of numerical noise, which always exists in theoretical computations. So, ICOHPs, which are negligible, will not be considered in the bonding analysis.

:::{caution}
So far LobsterEnv has only been tested for ICOHPs. ICOOP and ICOBI implementations are recently added.
:::

:::{seealso}
:class: dropdown

Please refer to the following sources for `ChemEnv` and `LobsterEnv` for more details on these algorithms.

* [D. Waroquiers, J. George, M. Horton, S. Schenk, K. A. Persson, G.-M. Rignanese, X. Gonze, G. Hautier, Acta Cryst B 2020, 76, 683–695.](https://scripts.iucr.org/cgi-bin/paper?S2052520620007994)
* [J. George, G. Petretto, A. Naik, M. Esters, A. J. Jackson, R. Nelson, R. Dronskowski, G.-M. Rignanese, G. Hautier, ChemPlusChem 2022, e202200123](https://chemistry-europe.onlinelibrary.wiley.com/doi/full/10.1002/cplu.202200123)
:::

## Evaluating bonding and anti-bonding percentages

LobsterPy follows following steps to evaluate the bonding and anti-bonding percentages at each symetrically independent sites:

1. All the relevant bonds are identfied for site in the structure using ICOHPs via LobsterEnv (bond labels from `ICOHPLIST.lobster` are also stored, along with ICOHP values).
2. Read the COHP data from `COHPCAR.lobster` corresponding to bond labels from step 1 and get summed COHPs data for each site seperately.
3. Perform numerical integration of the summed COHPs at each site to evaluate the bonding and antibonding integral and percentages.
4. One can also set the `start` parameter when initializing Analysis class object to set the start point of energy range below efermi for the intergration. (Optional and useful if one wants to look at specific energy range below efermi)

:::{note}
Ideally, the $(bonding-antibonding)$ integral values should equal to the summed ICOHP values at the each site. However, as it is a known fact, numerical integration are bound of have some error associated with it. Thus it can happen we might not see this corroborate very well. 

Also, it is important to note that the ICOHP values from `ICOHPLIST.lobster` consider the entire range below efermi. Thus if the `COHPCAR.lobster` are not computed for the entire energy range below efermi, then your results will also not corroborate to summed ICOHPs at the site. 

Another crucial factor which influences the results of the numerical integration is the number of points on which the integration is performed. Thus having a finer grid i.e setting `COHPSteps` to `2000` or in your LOBSTER run will help to get better estimates. From our experience setting `COHPSteps` to `10000` provided much consistent results. (Be mindful that the sizes of files generated from such setting can become quite large depending on the structure complexcity)
:::

Below is dummy visual depiction of evaluation of bonding percentage after steps 1 and 2 from COHP curve data.

![Bonding percentage evaluation](fundamentals_assets/bonding_percentage.png)

In similar way Antibonding percentage is evaluated at each site from the COHP curves.

## Automatic orbital-wise analysis implementation

LobsterPy follows following steps to evaluate the relevant oribitals contributiing in the bonds automatically. Step 1 to 3 are identical to the **Evaluating bonding and anti-bonding percentages**. Thus for sake of brevity we will not relist them here and only list the steps further. The steps mentioned below are repeated for each symetrically independent site:

4. Read the list of available orbitals from  `COHPCAR.lobster` at the site
5. Get summed cohps for each orbital using the bond label list (step 1)
6. Check if the total bonding and antibonding integral obtained in step 3 is greater than 0
7. Evaluate the bonding and antibonding integral for each orbital and store these values in separate lists of orbital pair names for bonding, bonding integral lists, orbital pair names for antibonding and antibonding integral list
8. Evaluate the orbital contribution percentages using the following formula:
$$\text{(Anti)  bonding contribution for each orbital}=\frac{\text{each orbital (anti)bonding integral}}{\text{sum(orbital (anti)bonding integral}}$$
9. If the orbital contribution is more than the `orbital_cutoff` set in the Analysis class object, include that orbital pair interaction as relevant (do this for bonding and antibonding separately using contribution percentages as evaluated in step 8.
10. Get max anti(bonding) contribution using contribution percentages evaluated in step 8 and add it to orbital summary dict (this is used in description text output generation). 

As mentioned in step 9, the results will vary a lot depending on the `orbital_cutoff` set when initializing the Analysis class object. Default value is set to 0.05 (i.e. 5%).

## Features curated for ML via Featurizer

### COHP based features

The below mentioned features are implemented based on the work of 

[R. Nelson, C. Ertural, P. C. Müller, R. Dronskowski, in Compr. Inorg. Chem. III, Elsevier, 2023, pp. 141–201.](https://linkinghub.elsevier.com/retrieve/pii/B9780128231449001205)

<h4><center>Average features</center></h4>

The $\overline{\mathrm{ICOHP}}$ can be interpreted as the average bond strength inside the given material.

$\overline{\mathrm{COHP}}(E) =\sum_i\left(w_i \cdot \operatorname{COHP}_i(E)\right)$ 

$w_i =\frac{\mathrm{ICOHP}_i}{\mathrm{ICOHP}_{\text {total}}}$

$\overline{\mathrm{ICOHP}}=\int_{-\infty}^{\varepsilon_{\mathrm{F}}} \overline{\mathrm{COHP}}(E) \mathrm{d}E $

The EIN is comparable to a coordination number

$\mathrm{EIN}=\frac{\mathrm{ICOHP}_{\text {total }}}{\overline{\mathrm{ICOHP}}} \cdot \frac{2}{N_{\text {Atoms }}}$


<h4><center>Moment based features and fingerprints </center></h4>

These features are inspired from similar set of features implemented for DOS in [pymatgen](https://github.com/materialsproject/pymatgen/tree/master) with some additional functionalities. Namely
1. [Center](https://github.com/materialsproject/pymatgen/blob/2ccbfa16cbffb1016501c18117e70a3f96a83de2/pymatgen/electronic_structure/dos.py#L833)
2. [Width](https://github.com/materialsproject/pymatgen/blob/2ccbfa16cbffb1016501c18117e70a3f96a83de2/pymatgen/electronic_structure/dos.py#L862)
3. [Skewness](https://github.com/materialsproject/pymatgen/blob/2ccbfa16cbffb1016501c18117e70a3f96a83de2/pymatgen/electronic_structure/dos.py#L889)
4. [Kurtosis](https://github.com/materialsproject/pymatgen/blob/2ccbfa16cbffb1016501c18117e70a3f96a83de2/pymatgen/electronic_structure/dos.py#L919)
5. [Fingerprints](https://github.com/materialsproject/pymatgen/blob/2ccbfa16cbffb1016501c18117e70a3f96a83de2/pymatgen/electronic_structure/dos.py#L1093)

For the case of COHPs, the $n^{th}$ moment is computed using following relation

$n^{th}_{moment} = \frac{\int (p^n \cdot \text{COHPs} \, dE)}{\int \text{COHPs} \, dE}$

The resulting $1^{st}$, $2^{nd}$, $3^{rd}$ and $4^{th}$ moment are named as center_COHP, width_COHP, skewness_COHP, kurtosis_COHP, respectively.

And Fingerprints are basically COHP data discretized into histograms that will allow to compute similarity between COHP curves of different materials numerically.

:::{note}
One can select the range of energy, type of interactions (bonding, antibonding, overall) one would like to compute these features.
:::

<h4><center>Summary stats features from LobsterPy condensed bonding analysis dicts</center></h4>

These features were first implemented as part of our Database publication : [A. A. Naik, C. Ertural, N. Dhamrait, P. Benner, J. George, Sci. Data 2023, 10, 610.](https://doi.org/10.1038/s41597-023-02477-5)

The source code can be found [here](https://github.com/naik-aakash/lobster-database-paper-analysis-scripts/blob/e274cc66fe5cdef981c9de0609eaba4368226c9a/ML_model/featurizer/lobster_featurizer.py#L33).

This is code have been intergrated in our Package and it is extended to also work for COBI/COOP condensed bonding analysis dicts and can also include orbital wise analysis summary stats as well if enabled.

### Charge based features

As Ionicity quantifies the strength of an ionic bond in the compound, there are several approaches to quantify it. Here we use the formulation as per [R. Nelson, C. Ertural, P. C. Müller, R. Dronskowski, in Compr. Inorg. Chem. III, Elsevier, 2023, pp. 141–201.](https://linkinghub.elsevier.com/retrieve/pii/B9780128231449001205), which use following relation:   

$I_{\text {Charges }}=\frac{1}{N_{\text {Atoms }}} \sum_i^{N_{\text {Atoms }}}\left(\frac{q_i}{v_{\text {eff }, i}}\right)$

It is possible to compute ionicity based on above mentioned formula using either the Mulliken or Löwdin charges obtained from LOBSTER run.